# Vector SurfacePDEs

## Vector-Laplace problem
We want to solve a problem of the form

$$
\begin{aligned}
          -2 \, \textbf{P} \, \textbf{div}_\Gamma (E_s(\textbf{u})) + \textbf{u} = & \, \textbf{f} 
          & & \text{on}~~ \Gamma,          
        \end{aligned}
$$

with $\textbf{P} \textbf{u} = \textbf{u}$ and $\Gamma := \{ \phi = 0 \}$. Here, $\textbf{P} = \textbf{I} - \textbf{n}\textbf{n}^T$ is the projection onto the tangential space and 

\begin{equation*}
\nabla_{\Gamma} \textbf{u} = \textbf{P} \nabla \textbf{u} \textbf{P}, \qquad E_s(\textbf{u}) = \frac{1}{2} (\nabla_{\Gamma} \textbf{u} + \nabla_{\Gamma}^T \textbf{u})
\end{equation*} 
are the surface gradient for vector-valued functions and the surface rate-of-strain tensor. Moreover, the surface divergence operators are defined as follows:

\begin{equation*}
\operatorname{div}_{\Gamma}\textbf{u} = \operatorname{tr}(\nabla_{\Gamma}\textbf{u}), \qquad \textbf{div}_{\Gamma}\textbf{A} = (\operatorname{div}_{\Gamma} (\textbf{e}_1^T \textbf{A}), \ldots, \operatorname{div}_{\Gamma}(\textbf{e}_n^T \textbf{A}))^T.
\end{equation*}

In [ ]:
# ngsolve stuff
from ngsolve import *
# basic xfem functionality
from xfem import *
# basic geometry features (for the background mesh)
from netgen.occ import *
# for isoparametric mapping
from xfem.lsetcurv import *
# pi
from math import pi

## Geometry approximation
First, we generate the background mesh of the domain and use a simplicial triangulation.

In [ ]:
cube = OCCGeometry(unit_cube.shape.Scale((0,0,0),3).Move((-1.5,-1.5,-1.5)), dim=3)
mesh = Mesh (cube.GenerateMesh(maxh=0.4, quad_dominated=False))
Draw(mesh)

#### Higher order geometry approximation with an isoparametric mapping
In order to get higher order convergence we can use the isoparametric mapping functionality of xfem.

We apply a mesh transformation technique in the spirit of isoparametric finite elements
![title](graphics/lsetcurv.jpg)

To compute the corresponding mapping we use the `LevelSetMeshAdaptation` class

In [ ]:
phi = Norm(CoefficientFunction((x,y,z)))-1
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=2)
deformation = lsetmeshadap.CalcDeformation(phi)
lset_approx = lsetmeshadap.lset_p1
mesh.SetDeformation(deformation)

## Weak formulation with penalty term for tangential flow

### Isoparametric TraceFE / CutFE space
For the discretization we use standard background FESpaces restricted to the tetrahedrons cut by $\Gamma_h$ and transformed to the deformed mesh:

\begin{equation*}
U_{h,\Theta}^k = \left\{ \textbf{u}_h \circ \Psi_h^{-1} \mid \textbf{u}_h \in (V_h^k)^3|_{ \Omega^{\Gamma}} \right\} \end{equation*}

In NGSolve we will simply take the space $(V_h^k)^3$, which is of order $k$, but mark the irrelevant dofs using the CutInfo-class:

In [ ]:
# FESpace 
Uhbase = VectorH1(mesh, order=2)
ci = CutInfo(mesh, lset_approx)

Uh = Compress(Uhbase,GetDofsOfElements(Uhbase,ci.GetElementsOfType(IF)))

# expressions of test and trial functions:
u,v = Uh.TnT()
# overwrite freedofs of VhG to mark only dofs that are involved in the cut problem
freedofs = Uh.FreeDofs()
# declare a grid function to store the solution
gfu = GridFunction(Uh)

### Isoparametric TraceFEM / CutFEM discretization

Find $\textbf{u}_h \in U_{h,\Theta}^k$ s.t.
    \begin{align*}
      \int_{\Gamma_h} \mathrm{tr}(\tilde{E}_s(\textbf{u}_h) \tilde{E}_s(\textbf{v}_h)) &+ \textbf{u}_h \cdot \textbf{v}_h \,ds_h + \eta \int_{\Gamma_h} (\textbf{u}_h \cdot \textbf{n}_h)  (\textbf{v}_h \cdot \textbf{n}_h) \,ds_h  + \rho\int_{\Omega_{\Theta}^\Gamma}(\nabla \textbf{u}_h \textbf{n}_h)\cdot(\nabla\textbf{v}_h \textbf{n}_h)\,dx \\
      &= (\mathbf{f}, \textbf{v}_h)_{0,\Gamma} \qquad \text{ for all }\textbf{v}_h\in U_{h,\Theta}^k.
    \end{align*}
with $\tilde{E}_s(\textbf{u}) = \frac{1}{2} (\nabla_{\Gamma_h} \textbf{u} + \nabla_{\Gamma_h}^T \textbf{u})$, $\eta \sim h^{-2}$ and $h\lesssim \rho \lesssim h^{-1}$.

In [ ]:
# declare the integration domains
ds = dCut(levelset=lset_approx, domain_type=IF, definedonelements=ci.GetElementsOfType(IF),deformation=deformation)
dX = dx(definedonelements=ci.GetElementsOfType(IF),deformation=deformation)

#### helper coefficients / parameters

In [ ]:
# calculate normal vector n, mesh size h and parameters eta and rho
n = 1.0/Norm(grad(lset_approx)) * grad(lset_approx)
h = specialcf.mesh_size
eta = 100.0/(h*h)
rho = 1.0/h

# define solution and right-hand side
functions = {
           "extvsol1" : ((-y-z)*x + y*y + z*z),
           "extvsol2" : ((-x-z)*y + x*x + z*z),
           "extvsol3" : ((-x-y)*z + x*x + y*y),
           "rhs1" : -((y + z) * x - y * y - z * z) * (x * x + y * y + z * z + 1) / (x * x + y * y + z * z),
           "rhs2" : ((-x - z) * y + x * x + z * z) * (x * x + y * y + z * z + 1) / (x * x + y * y + z * z),
           "rhs3" : ((-x - y) * z + x * x + y * y) * (x * x + y * y + z * z + 1) / (x * x + y * y + z * z),
}
uSol = CoefficientFunction((functions["extvsol1"],functions["extvsol2"],functions["extvsol3"]))
rhs = CoefficientFunction((functions["rhs1"],functions["rhs2"],functions["rhs3"]))

The bi- and linear forms:
    \begin{align*}
      \int_{\Gamma_h} \mathrm{tr}(\tilde{E}_s(\textbf{u}_h) \tilde{E}_s(\textbf{v}_h)) & + \textbf{u}_h \cdot \textbf{v}_h \,ds_h + \eta \int_{\Gamma_h} (\textbf{u}_h \cdot \textbf{n}_h)  (\textbf{v}_h \cdot \textbf{n}_h) \,ds_h  + \rho\int_{\Omega_{\Theta}^\Gamma}(\nabla \textbf{u}_h \textbf{n}_h)\cdot(\nabla\textbf{v}_h \textbf{n}_h)\,dx \\ &= (\mathbf{f}, \textbf{v}_h)_{0,\Gamma} \qquad \text{for all }\textbf{v}_h\in U_{h,\Theta}^k.
    \end{align*}

In [ ]:
# define projection matrix
Pmat = Id(3) - OuterProduct(n,n)

# bilinear forms:
a = BilinearForm(Uh, symmetric = True)
a += (InnerProduct(Pmat*Sym(grad(u))*Pmat,Pmat*Sym(grad(v))*Pmat))*ds
a += u * v * ds
a += (eta*((u * n) * (v * n))) * ds
a += rho * ((grad(u)*n) * (grad(v)*n)) * dX
a.Assemble()

f = LinearForm(Uh)
f += rhs * v * ds
f.Assemble()

We can now solve the problem:

In [ ]:
gfu.vec[:] = 0.0
c = a.mat.CreateSmoother(freedofs)
inv = CGSolver(a.mat, c, maxsteps=300)
gfu.vec.data = inv * f.vec
print("CG with Jac. preconditioner, steps:", inv.GetSteps())
    
l2error = sqrt(Integrate(InnerProduct(gfu-uSol, gfu-uSol)*ds, mesh))
print ("l2error : ", l2error)

### Visualization
Create vtk-output file to be read by Paraview.

In [ ]:
vtk = VTKOutput(ma=mesh,
                coefs=[lset_approx,deformation,gfu,uSol,rhs],
                names=["P1-levelset","deform","uh","uex","rhs"],
                filename="vectorlaplacepenalty3d",subdivision=2)
vtk.Do()

<center>
![title](graphics/tracefem_vector1.png)
</center>

In [ ]:
%%bash
ls -al vectorlaplacepenalty3d.vtu

In [ ]:
#%%bash
#paraview vectorlaplacepenalty3d.vtk

## Weak formulation with Lagrange-multiplier term for tangential flow

### Isoparametric TraceFE / CutFE space
For the discretization we use standard background FESpaces restricted to the tetrahedrons cut by $\Gamma_h$ and transformed to the deformed mesh:

\begin{equation*}
U_{h,\Theta}^k = \left\{ \textbf{u}_h \circ \Psi_h^{-1} \mid \textbf{u}_h \in (V_h^k)^3|_{ \Omega^{\Gamma}} \right\}, \qquad M_{h,\Theta}^k = \left\{ \lambda_h \circ \Psi_h^{-1} \mid \lambda_h \in V_h^k|_{ \Omega^{\Gamma}} \right\}
\end{equation*}

In NGSolve we will simply take the space $(V_h^k)^3$ for the solution and $V_h^k$ for the lagrange multiplier but mark the irrelevant dofs using the CutInfo-class:

In [ ]:
# FESpace 
Vhbase = VectorH1(mesh, order=2)
Mhbase = H1(mesh, order=1)
#ci = CutInfo(mesh, lset_approx)

VhG = Compress(Vhbase,GetDofsOfElements(Vhbase,ci.GetElementsOfType(IF)))
Mh = Compress(Mhbase,GetDofsOfElements(Mhbase,ci.GetElementsOfType(IF)))

Qh = VhG * Mh
(u,p), (v,q) = Qh.TnT()

# overwrite freedofs of VhG to mark only dofs that are involved in the cut problem
freedofs = Qh.FreeDofs()
# declare a grid function to store the solution
gfu = GridFunction(Qh)

### Isoparametric TraceFEM / CutFEM discretization 
 Find $(\textbf{u}_h,p_h) \in U_{h,\Theta}^k \times M_{h,\Theta}^l$ s.t.
    \begin{align*}
      A_h(\textbf{u}_h,\textbf{v}_h) + B_h(\textbf{v}_h,p_h) &= (\mathbf{f}, \textbf{v}_h)_{0,\Gamma} &&\text{for all }\textbf{v}_h\in U_{h,\Theta}^k \\[1ex]
      B_h(\textbf{u}_h,q_h) &= 0 &&\text{for all } q_h\in M_{h,\Theta}^l
    \end{align*}
    with bilinear forms using stabilization
    \begin{align*}
      A_h(\textbf{u}_h,\textbf{v}_h) &:= \int_{\Gamma_h} \mathrm{tr}(\tilde{E}_s(\textbf{u}_h) \tilde{E}_s(\textbf{v}_h)) + \textbf{u}_h \cdot \textbf{v}_h \,ds_h + \rho\int_{\Omega_{\Theta}^\Gamma}(\nabla \textbf{u}_h \textbf{n})\cdot(\nabla\textbf{v}_h \textbf{n})\,dx,\\[1ex]
      B_h(\textbf{u}_h,q_h) &:= (\textbf{u}_h\cdot\textbf{n},q_h)_{0,\Gamma} + 
        \rho \int_{\Omega_{\Theta}^\Gamma}(\textbf{n}^T\nabla \textbf{u}_h\textbf{n})(\textbf{n}\cdot\nabla q_h)\,dx, 
    \end{align*}
    with $\tilde{E}_s(\textbf{u}) = \frac{1}{2} (\nabla_{\Gamma_h} \textbf{u} + \nabla_{\Gamma_h}^T \textbf{u})$ and $h\lesssim \rho \lesssim h^{-1}$.

In [ ]:
# declare the integration domains
lset_if = {"levelset": lset_approx, "domain_type": IF, "subdivlvl": 0}
def SurfaceIntegral(form):
    return SymbolicBFI(levelset_domain = lset_if , form = form.Compile(), definedonelements=ci.GetElementsOfType(IF))
def ActiveMeshIntegral(form):
    return SymbolicBFI(form = form.Compile(), definedonelements=ci.GetElementsOfType(IF))

#### helper coefficients / parameters:

In [ ]:
# calculate normal vector n, mesh size h and parameters eta and rho
n = 1.0/sqrt(InnerProduct(grad(lset_approx),grad(lset_approx))) * grad(lset_approx)
h = specialcf.mesh_size
rho = 1.0*h

# define solution and right-hand side
functions = {
           "extvsol1" : ((-y-z)*x + y*y + z*z),
           "extvsol2" : ((-x-z)*y + x*x + z*z),
           "extvsol3" : ((-x-y)*z + x*x + y*y),
           "rhs1" : -((y + z) * x - y * y - z * z) * (x * x + y * y + z * z + 1) / (x * x + y * y + z * z),
           "rhs2" : ((-x - z) * y + x * x + z * z) * (x * x + y * y + z * z + 1) / (x * x + y * y + z * z),
           "rhs3" : ((-x - y) * z + x * x + y * y) * (x * x + y * y + z * z + 1) / (x * x + y * y + z * z),
}
uSol = CoefficientFunction((functions["extvsol1"],functions["extvsol2"],functions["extvsol3"]))
rhs = CoefficientFunction((functions["rhs1"],functions["rhs2"],functions["rhs3"]))

# define projection matrix
Pmat = Id(3) - OuterProduct(n,n)

bilinear form
    \begin{align*}
      A_h(\textbf{u}_h,\textbf{v}_h) &:= \int_{\Gamma_h} \mathrm{tr}(E_s(\textbf{u}_h) E_s(\textbf{v}_h)) + \textbf{u}_h \cdot \textbf{v}_h \,ds_h + \rho\int_{\Omega_{\Theta}^\Gamma}(\nabla \textbf{u}_h \textbf{n})\cdot(\nabla\textbf{v}_h \textbf{n})\,dx,
    \end{align*}

In [ ]:
a = BilinearForm(Qh, symmetric = False)
a += (InnerProduct(Pmat*Sym(grad(u))*Pmat,Pmat*Sym(grad(v))*Pmat))*ds
a += u * v * ds
a += rho* ((grad(u)*n) * (grad(v)*n)) * dX

bilinear form
    \begin{align*}
      B_h(\textbf{u}_h,q_h) &:= (\textbf{u}_h\cdot\textbf{n},q_h)_{0,\Gamma} + 
        \rho \int_{\Omega_{\Theta}^\Gamma}(\textbf{n}^T\nabla \textbf{u}_h\textbf{n})(\textbf{n}\cdot\nabla q_h)\,dx, 
    \end{align*}

In [ ]:
a += (p * (v * n) + q * (u * n)) * ds
a += (rho* ((n*grad(p)) * (n*(grad(v)*n))+ (n*grad(q)) * (n*(grad(u)*n)))) * dX
a.Assemble()

linear form

In [ ]:
f = LinearForm(Qh)
f += rhs * v * ds
f.Assemble()

We can now solve the problem (recall that freedofs only marks relevant dofs):

In [ ]:
gfu.vec[:] = 0.0
gfu.vec.data = a.mat.Inverse(freedofs) * f.vec
    
uNum = gfu.components[0]
l2error = sqrt(Integrate(InnerProduct(Pmat*(uNum-uSol), Pmat*(uNum-uSol))*ds, mesh))
print ("l2error : ", l2error)

### Visualization with VTK
Create vtk-output file to be read by Paraview.

In [ ]:
mesh.UnsetDeformation()
vtk = VTKOutput(ma=mesh,
                coefs=[lset_approx,deformation,uNum,uSol,rhs],
                names=["P1-levelset","deform","u","uSol","rhs"],
                filename="vectorlaplacelagrange3d",subdivision=2)
vtk.Do()

<center>
![title](graphics/tracefem_vector2.png)
</center>

In [ ]:
%%bash
ls -al vectorlaplacelagrange3d.vtu

In [ ]:
#%%bash
#paraview vectorlaplacelagrange3d.vtk